In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('/content/name_entity.csv',encoding='latin1')

In [3]:
df.fillna(method='ffill',inplace=True)

In [4]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [5]:
words=list(set(df['Word']))
words[:10]

['re-sales',
 'prevention',
 'decreed',
 'contributes',
 'Guards',
 'church',
 'pulling',
 'secretive',
 'colonists',
 'reservists']

In [6]:
n_words=len(words)
n_words

9578

In [7]:
tags=list(set(df['Tag']))
tags[:10]

['I-geo',
 'B-nat',
 'B-art',
 'I-art',
 'O',
 'B-geo',
 'B-org',
 'I-eve',
 'B-per',
 'I-tim']

In [8]:
n_tags=len(tags)

In [9]:
n_tags

17

In [10]:
class GetSentence(object):
  def __init__ (self,data):
    self.n_sentence=1
    self.data=data
    self.empty=False
    function=lambda d:[(w,p,t) for w,p,t in zip(d['Word'].values.tolist(),
                                                d['POS'].values.tolist(),
                                                d['Tag'].values.tolist())]
                    
    self.group_sent=self.data.groupby('Sentence #').apply(function)
    self.all_sentences=[d for d in self.group_sent]

In [11]:
get=GetSentence(df)
sentences=get.all_sentences

In [12]:
maxl=max([len(s) for s in sentences])
print('Maximum length of sentence:',maxl)

Maximum length of sentence: 70


In [13]:
w_index={w:i for i,w in enumerate(words)}
t_index={t:j for j,t in enumerate(tags)}

In [14]:
from keras.preprocessing.sequence import pad_sequences

In [15]:
X=[[w_index[w[0]] for w in s] for s in sentences]
X=pad_sequences(sequences=X,maxlen=maxl,padding='pre')

In [16]:
Y=[[t_index[t[2]] for t in s] for s in sentences]
Y=pad_sequences(sequences=Y,maxlen=maxl,padding='pre')

In [17]:
Y

array([[0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 5, 0, 4],
       ...,
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 4, 4],
       [0, 0, 0, ..., 4, 5, 4]], dtype=int32)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42)

In [20]:
from keras.utils import to_categorical

In [21]:
y_train=to_categorical(y_train)
print(y_train.shape)

(2494, 70, 17)


In [22]:
y_test=to_categorical(y_test)
print(y_test.shape)

(1069, 70, 17)


In [23]:
y_test=np.array(y_test)
y_train=np.array(y_train)

In [24]:
from keras import Model,Input
from keras.layers import LSTM,Dense,Embedding,SpatialDropout1D,Bidirectional,TimeDistributed

In [25]:
input=Input(shape=(70))
model=Embedding(input_dim=n_words,output_dim=70,input_length=140)(input)
model=SpatialDropout1D(0.25)(model)
model=Bidirectional(LSTM(200,activation='relu',return_sequences=True,recurrent_dropout=0.1))(model)
output=TimeDistributed(Dense(n_tags,activation='softmax'))(model)
model=Model(input,output)

In [26]:
model.compile(optimizer='adam',metrics='accuracy',loss='categorical_crossentropy')

In [27]:
model.fit(X_train,y_train,batch_size=64,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
39/39 [==============================] - 41s 921ms/step - loss: 2.0757 - accuracy: 0.6722 - val_loss: 0.4633 - val_accuracy: 0.8949
Epoch 2/100
39/39 [==============================] - 35s 904ms/step - loss: 0.4177 - accuracy: 0.9079 - val_loss: 0.3505 - val_accuracy: 0.9147
Epoch 3/100
39/39 [==============================] - 35s 907ms/step - loss: 0.3215 - accuracy: 0.9248 - val_loss: 0.3059 - val_accuracy: 0.9305
Epoch 4/100
39/39 [==============================] - 35s 906ms/step - loss: 0.2794 - accuracy: 0.9382 - val_loss: 0.2857 - val_accuracy: 0.9393
Epoch 5/100
39/39 [==============================] - 35s 907ms/step - loss: 0.2519 - accuracy: 0.9466 - val_loss: 0.2605 - val_accuracy: 0.9478
Epoch 6/100
39/39 [==============================] - 35s 905ms/step - loss: 0.2208 - accuracy: 0.9519 - val_loss: 0.2062 - val_accuracy: 0.9506
Epoch 7/100
39/39 [==============================] - 35s 908ms/step - loss: 0.1645 - accuracy: 0.9558 - val_loss: 0.1428 - val_accuracy:

#lets predict

In [33]:
i=np.random.randint(0,X_test.shape[0])
p=model.predict(np.array([X_test[i]]))
p=np.argmax(p,axis=-1)
y_true=np.argmax(np.array(y_test),axis=-1)[i]
print("{:15} ({:5}): {}".format('Word',"True",'PREDICTED BY OUR MODEL'))
print("__"*20)
for w ,true,pred in zip(X_test[i],y_true,p[0]):
  print("{:15}{}\t {}".format(words[w-1],tags[true],tags[pred]))

Word            (True ): PREDICTED BY OUR MODEL
________________________________________
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       I-geo	 I-geo
refugees       